# Analyze the Voltage and Current Dynamics of a CUBA LIF Neuron and choose appropriate values for each use case

### Check WD (change if necessary) and file loading

In [1]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/dynamics_analysis" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/dynamics_analysis"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())
    

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/dynamics_analysis
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/dynamics_analysis


In [2]:
from du_dv_analysis import voltage_pred, CUBADynamicsResult
import numpy as np

## Channel Burst Detection 
Let's find the optimal parameters for the channel burst detection. we will **consider a network burst any sequence of 2 spikes that occurs within 10 ms**.

Therefore, in the worst case scenario, the 2 spikes are separated by 10 ms.

To find the optimal parameters, we will test different combinations of the parameters: `spike_weight`, `du`, and `dv` and we will see the resulting voltage in the worst case scenario that we still consider as a burst (10 ms). To find the optimal parameters, we will also calculate the voltage if the spike occurs 1 time step delayed and choose the parameters where:
1. The voltage is above the threshold in the worst case scenario that is still considered a burst.
2. The voltage is below the threshold if the spike occurs 1 time step delayed.

In [4]:
# Fixed Parameters
PADDING_TIME = 20               # 20ms padding time (to allow the voltage to reach the max value)
sim_time = 10 + PADDING_TIME    # 10ms simulation time (10 time steps)
num_spikes = 2                  # Number of spikes
spike_times = [0, 10]           # Let's consider the spikes are with maximum spacing
v_th = 1.0                      # Threshold voltage
recording_times = [10]

# Explorable Parameters
spike_weights = [0.05 * i for i in range(11)]
du_vals = [0.05 * i for i in range(1, 20)]
dv_vals = [0.05 * i for i in range(1, 20)]

# Variables to store the results
results_cond1 = []

# Run the simulation
for spike_weight in spike_weights:
    for du in du_vals:
        for dv in dv_vals:
            voltage_vals = voltage_pred(sim_time, du, dv, spike_times, recording_times, spike_weight)
            # print(f"bef_v: {bef_v}, v: {v}, after_v: {after_v}")

            bef_v, v, max_v = voltage_vals[0]

            currResult = CUBADynamicsResult(spike_weight, du, dv, bef_v, v, max_v)
            results_cond1.append(currResult)

In [3]:
# Show the results
print(results_cond1)

print("Number of Results Cond 1: ", len(results_cond1))

[{'spike_weight': 0.0, 'du': 0.05, 'dv': 0.05, 'before v': 0.0, 'spike v': 0.0, 'max v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.1, 'before v': 0.0, 'spike v': 0.0, 'max v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.15000000000000002, 'before v': 0.0, 'spike v': 0.0, 'max v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.2, 'before v': 0.0, 'spike v': 0.0, 'max v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.25, 'before v': 0.0, 'spike v': 0.0, 'max v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.30000000000000004, 'before v': 0.0, 'spike v': 0.0, 'max v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.35000000000000003, 'before v': 0.0, 'spike v': 0.0, 'max v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.4, 'before v': 0.0, 'spike v': 0.0, 'max v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.45, 'before v': 0.0, 'spike v': 0.0, 'max v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.5, 'before v': 0.0, 'spike v': 0.0, 'max v': 0.0}, {'spike_weight': 0

In [4]:
# Find the results where the Voltage is above 1.0 (i.e. a spike occurs
relevant_results1 = [result for result in results_cond1 if (
    result.bef_spike_v <= v_th 
    # and result.spike_v >= v_th 
    and result.max_spike_v >= v_th
    )]

# Print number of relevant results
print(f"Number of relevant results for Cond. 1: {len(relevant_results1)}")

# Sort the relevant results from lowest voltage to highest
relevant_results1 = sorted(relevant_results1, key=lambda item: item.max_spike_v, reverse=False)

# Show the relevant results
print(relevant_results1)

Number of relevant results for Cond. 1: 173
[{'spike_weight': 0.30000000000000004, 'du': 0.05, 'dv': 0.4, 'before v': 0.5080202756900388, 'spike v': 0.7844332471855369, 'max v': 1.0043979321210597}, {'spike_weight': 0.30000000000000004, 'du': 0.4, 'dv': 0.05, 'before v': 0.508020275690039, 'spike v': 0.7844332471855371, 'max v': 1.0043979321210597}, {'spike_weight': 0.35000000000000003, 'du': 0.15000000000000002, 'dv': 0.25, 'before v': 0.49196311370937507, 'spike v': 0.7878783768012843, 'max v': 1.016931025310691}, {'spike_weight': 0.35000000000000003, 'du': 0.25, 'dv': 0.15000000000000002, 'before v': 0.49196311370937496, 'spike v': 0.7878783768012843, 'max v': 1.0169310253106911}, {'spike_weight': 0.4, 'du': 0.15000000000000002, 'dv': 0.30000000000000004, 'before v': 0.4496716785085938, 'spike v': 0.7935199366923047, 'max v': 1.0195775800667901}, {'spike_weight': 0.4, 'du': 0.30000000000000004, 'dv': 0.15000000000000002, 'before v': 0.4496716785085937, 'spike v': 0.7935199366923047,

### Test the dynamics, but now with 1 more time step between the spikes

In [5]:
# Fixed Parameters
spike_times = [0, 11]           # Let's consider the spikes are with maximum spacing
recording_times = [11]

# Variables to store the results
results_cond2 = []

# Run the simulation
for spike_weight in spike_weights:
    for du in du_vals:
        for dv in dv_vals:
            voltage_vals2 = voltage_pred(sim_time, du, dv, spike_times, recording_times, spike_weight)

            bef_v, v, max_v = voltage_vals2[0]
            # print(f"bef_v: {bef_v}, v: {v}, after_v: {after_v}")

            currResult = CUBADynamicsResult(spike_weight, du, dv, bef_v, v, max_v)
            results_cond2.append(currResult)

In [6]:
# Find the results where the Voltage is below 1.0
relevant_results2 = [result for result in results_cond2 if (
    result.bef_spike_v <= v_th and 
    result.spike_v <= v_th and
    result.max_spike_v <= v_th
    )]

# Print number of relevant results
print(f"Number of relevant results Cond. 2: {len(relevant_results2)}")

# Sort the relevant results from lowest voltage to highest
relevant_results2 = sorted(relevant_results2, key=lambda item: item.spike_v, reverse=True)

# Show the relevant results
print(relevant_results2)

Number of relevant results Cond. 2: 3740
[{'spike_weight': 0.5, 'du': 0.75, 'dv': 0.05, 'before v': 0.40628560989848617, 'spike v': 0.8859714486128514, 'max v': 0.9666729059845312}, {'spike_weight': 0.5, 'du': 0.05, 'dv': 0.75, 'before v': 0.4062856098984861, 'spike v': 0.8859714486128513, 'max v': 0.9666729059845312}, {'spike_weight': 0.5, 'du': 0.05, 'dv': 0.8, 'before v': 0.3792000478643064, 'spike v': 0.8602400557110911, 'max v': 0.9172280549735365}, {'spike_weight': 0.5, 'du': 0.8, 'dv': 0.05, 'before v': 0.37920004786430644, 'spike v': 0.8602400557110911, 'max v': 0.9172280549735364}, {'spike_weight': 0.45, 'du': 0.05, 'dv': 0.65, 'before v': 0.42659282802054177, 'spike v': 0.8552675313315965, 'max v': 0.9766309238707631}, {'spike_weight': 0.45, 'du': 0.65, 'dv': 0.05, 'before v': 0.4265928280205419, 'spike v': 0.8552675313315965, 'max v': 0.976630923870763}, {'spike_weight': 0.5, 'du': 0.8500000000000001, 'dv': 0.05, 'before v': 0.35550005713217764, 'spike v': 0.8377250547080566

### See if the intersection between both solutions is not empty

In [7]:
relevant_results = []

for rel_result in relevant_results1:
    if rel_result in relevant_results2:
        # Get the index of the relevant result
        index = relevant_results2.index(rel_result)
        delayed_result = relevant_results2[index]

        relevant_results.append({
            "spike_weight": rel_result.spike_weight, "du": rel_result.du, "dv": rel_result.dv, 
            "max_v_on_time": rel_result.max_spike_v, "max_v_delayed": delayed_result.max_spike_v,
            "spike_v_on_time": rel_result.spike_v, "spike_v_delayed": delayed_result.spike_v,
            "bef_v_on_time": rel_result.bef_spike_v, "bef_v_delayed": delayed_result.bef_spike_v,
        })

        # print(f"Parameters: {rel_result}")
        # print(f"On Time Results: {rel_result.to_dict()}")
        # print(f"Voltage delayed: {relevant_results2[index].to_dict()}\n")

# Sort the relevant results by lowest voltage later_v_on_time
relevant_results = sorted(relevant_results, key=lambda item: item["max_v_on_time"], reverse=False)

for rel_result in relevant_results:
    print(rel_result)

{'spike_weight': 0.30000000000000004, 'du': 0.05, 'dv': 0.4, 'max_v_on_time': 1.0043979321210597, 'max_v_delayed': 0.9837662530072946, 'spike_v_on_time': 0.7844332471855369, 'spike_v_delayed': 0.76129997599426, 'bef_v_on_time': 0.5080202756900388, 'bef_v_delayed': 0.4844332471855369}
{'spike_weight': 0.30000000000000004, 'du': 0.4, 'dv': 0.05, 'max_v_on_time': 1.0043979321210597, 'max_v_delayed': 0.9837662530072949, 'spike_v_on_time': 0.7844332471855371, 'spike_v_delayed': 0.7612999759942602, 'bef_v_on_time': 0.508020275690039, 'bef_v_delayed': 0.48443324718553704}
{'spike_weight': 0.35000000000000003, 'du': 0.15000000000000002, 'dv': 0.25, 'max_v_on_time': 1.016931025310691, 'max_v_delayed': 0.9785676533188277, 'spike_v_on_time': 0.7878783768012843, 'spike_v_delayed': 0.7369789178923282, 'bef_v_on_time': 0.49196311370937507, 'bef_v_delayed': 0.4378783768012842}
{'spike_weight': 0.35000000000000003, 'du': 0.25, 'dv': 0.15000000000000002, 'max_v_on_time': 1.0169310253106911, 'max_v_dela

Looking at the results, we can see that the best parameters combination to detect the channel bursts with the above conditions are:

(**Dense Layer Weight**, **du**, **dv**):
- `(0.3, 0.05, 0.4)` or `(0.3, 0.4, 0.05)`
Followed by:
- `(0.35, 0.15, 0.25)` or `(0.35, 0.25, 0.15)`

Let's choose the parameters:
- `spike_weight = 0.3`
- `du = 0.4`
- `dv = 0.05`
as the optimal parameters and test it in a LAVA LIF Layer.

---

## Channel Burst Detection (4 Spikes)
Let's find the optimal parameters for the channel burst detection. we will **consider a network burst any sequence of 4 spikes that occurs within 20 ms**.

Therefore, in the worst case scenario, the 4 spikes are separated by 20 ms.

To find the optimal parameters, we will test different combinations of the parameters: `spike_weight`, `du`, and `dv` and we will see the resulting voltage in the worst case scenario that we still consider as a burst (20 ms). To find the optimal parameters, we will also calculate the voltage if the spike occurs 1 time step delayed and choose the parameters where:
1. The voltage is above the threshold in the worst case scenario that is still considered a burst.
2. The voltage is below the threshold if the spike occurs 1 time step delayed.

In [8]:
# Fixed Parameters
PADDING_TIME = 20               # 20ms padding time (to allow the voltage to reach the max value)
MAX_SPACING = 20                # Maximum spacing between 1st and last spike of the burst
sim_time = MAX_SPACING + PADDING_TIME    # 10ms simulation time (10 time steps)
num_spikes = 4                  # Number of spikes
v_th = 1.0                      # Threshold voltage

# Explorable Parameters
consec_spikes_spacing = 4 # None   # Space between consecutive spikes (Start from 1 and increment until the result space is > 0)
two_phase_consec_spacing = None      # Space between consecutive spikes when using stimulating in 2phases (Start from 1 and increment until the result space is > 0)

In [9]:
from du_dv_analysis import get_intersection_parameter_combinations

result_cb_4spikes = get_intersection_parameter_combinations(PADDING_TIME, MAX_SPACING, sim_time, num_spikes, consec_spikes_spacing=consec_spikes_spacing, 
                                                            two_consec_spacing=two_phase_consec_spacing, v_th=v_th, verbose=True)

Number of Results Cond 1:  6498
Number of relevant results for Cond. 1: 787
[{'spike_weight': 0.2, 'du': 0.2, 'dv': 0.15000000000000002, 'before v': 0.7875256349581612, 'spike v': 0.9224417657458587, 'max v': 1.0004835441128594}, {'spike_weight': 0.2, 'du': 0.15000000000000002, 'dv': 0.2, 'before v': 0.7875256349581612, 'spike v': 0.9224417657458587, 'max v': 1.0004835441128597}, {'spike_weight': 0.8500000000000001, 'du': 0.45, 'dv': 0.45, 'before v': 0.16991301555069763, 'spike v': 0.956595325281043, 'max v': 1.0008561706050614}, {'spike_weight': 0.8500000000000001, 'du': 0.25, 'dv': 0.9, 'before v': 0.20200308410851597, 'spike v': 1.0015023980813955, 'max v': 1.0015023980813955}, {'spike_weight': 0.8500000000000001, 'du': 0.9, 'dv': 0.25, 'before v': 0.20200308410851595, 'spike v': 1.0015023980813955, 'max v': 1.0015023980813955}, {'spike_weight': 0.8500000000000001, 'du': 0.30000000000000004, 'dv': 0.65, 'before v': 0.21684871100331055, 'spike v': 1.002341334233905, 'max v': 1.00234

In [10]:
for result in result_cb_4spikes:
    print(result)

{'spike_weight': 0.2, 'du': 0.45, 'dv': 0.05, 'max_v_on_time': 1.006320780288703, 'max_v_delayed': 0.9737942575953025, 'max_v_consecutive': 0.9707599563174022, 'max_v_two_phase': 0.9737942575953025, 'spike_v_on_time': 0.9293844428681785, 'bef_v_on_time': 0.7645178242629348}
{'spike_weight': 0.2, 'du': 0.05, 'dv': 0.45, 'max_v_on_time': 1.0063207802887033, 'max_v_delayed': 0.9737942575953026, 'max_v_consecutive': 0.9707599563174021, 'max_v_two_phase': 0.9737942575953026, 'spike_v_on_time': 0.9293844428681786, 'bef_v_on_time': 0.7645178242629348}
{'spike_weight': 0.30000000000000004, 'du': 0.05, 'dv': 0.7000000000000001, 'max_v_on_time': 1.0183338166348954, 'max_v_delayed': 0.9861730319954936, 'max_v_consecutive': 0.9902741192138672, 'max_v_two_phase': 0.9861730319954936, 'spike_v_on_time': 0.9771727677127213, 'bef_v_on_time': 0.7127443614245831}
{'spike_weight': 0.30000000000000004, 'du': 0.7000000000000001, 'dv': 0.05, 'max_v_on_time': 1.0183338166348954, 'max_v_delayed': 0.98617303199

Looking at the results, we can see that the best parameters combination to detect the channel bursts with the above conditions are:

(**Dense Layer Weight**, **du**, **dv**):
- `(0.2, 0.45, 0.05)` or `(0.2, 0.05, 0.45)`
Followed by:
- `(0.3, 0.05, 0.45)` or `(0.3, 0.7, 0.05)`

Let's choose the parameters:
- `spike_weight = 0.2`
- `du = 0.45`
- `dv = 0.05`
  
as the optimal parameters and test it in a LAVA LIF Layer.

---

## Network Burst Detection
Let's find the optimal parameters for the network burst detection:
- we will **consider a channel burst a neuronal activity where 4 spikes occur within 20ms in the same channel**.
- we will **consider a network burst a neuronal activity where 3 channels spike in a 20ms time frame**.
- Network IBI = 40ms.

Therefore, in the worst case scenario, the 3 spikes corresponding to channel bursts are separated by at most 20ms.

To find the optimal parameters, we will test different combinations of the parameters: `spike_weight`, `du`, and `dv` and we will see the resulting voltage in the worst case scenario that we still consider as a burst (20 ms). To find the optimal parameters, we will also calculate the voltage if:
1. The last spike occurs 1 time step delayed
2. The spikes are consecutively fed to the network, but 1 less than the designated.

and choose the parameters where:

3. The voltage is above the threshold in the worst case scenario that is still considered a burst.
4. The voltage is below the threshold if the spike occurs 1 time step delayed.
5. The voltage is below the threshold if the spikes are consecutively fed to the network, but 1 less than the designated.

In [5]:
# Fixed Parameters
PADDING_TIME = 20               # 20ms padding time (to allow the voltage to reach the max value)
CAUSALITY_WINDOW = 20                # Maximum spacing between 1st and last spike of the burst
sim_time = CAUSALITY_WINDOW + PADDING_TIME    # 10ms simulation time (10 time steps)
num_spikes = 3                  # Number of spikes
v_th = 1.0                      # Threshold voltage

# Explorable Parameters
consec_spikes_spacing = 1 # None   # Space between consecutive spikes (Start from 1 and increment until the result space is > 0)
two_phase_consec_spacing = None      # Space between consecutive spikes when using stimulating in 2phases (Start from 1 and increment until the result space is > 0)

In [6]:
from du_dv_analysis import get_intersection_parameter_combinations

result_nb_3spikes = get_intersection_parameter_combinations(PADDING_TIME, CAUSALITY_WINDOW, sim_time, num_spikes, consec_spikes_spacing=consec_spikes_spacing, 
                                                            two_consec_spacing=two_phase_consec_spacing, v_th=v_th, verbose=True)

Number of Results Cond 1:  6859
Number of relevant results for Cond. 1: 959
[{'spike_weight': 0.8, 'du': 0.15000000000000002, 'dv': 0.9500000000000001, 'before v': 0.2356339354251393, 'spike v': 1.0002888451114464, 'max v': 1.0002888451114464}, {'spike_weight': 0.8, 'du': 0.9500000000000001, 'dv': 0.15000000000000002, 'before v': 0.23563393542513925, 'spike v': 1.0002888451114464, 'max v': 1.0002888451114464}, {'spike_weight': 0.65, 'du': 0.15000000000000002, 'dv': 0.65, 'before v': 0.3062882539479665, 'spike v': 0.9103629469081923, 'max v': 1.0013147807403109}, {'spike_weight': 0.65, 'du': 0.65, 'dv': 0.15000000000000002, 'before v': 0.30628825394796655, 'spike v': 0.9103629469081924, 'max v': 1.0013147807403109}, {'spike_weight': 0.8500000000000001, 'du': 0.4, 'dv': 0.45, 'before v': 0.060244799227996776, 'spike v': 0.8883053418821388, 'max v': 1.001670359419221}, {'spike_weight': 0.8500000000000001, 'du': 0.45, 'dv': 0.4, 'before v': 0.06024479922799679, 'spike v': 0.888305341882138

In [7]:
for result in result_nb_3spikes:
    print(result)

{'spike_weight': 0.5, 'du': 0.9500000000000001, 'dv': 0.05, 'max_v_on_time': 1.005200954758094, 'max_v_delayed': 0.9799409070201917, 'max_v_consecutive': 0.9999999999999999, 'max_v_two_phase': 0.9799409070201917, 'spike_v_on_time': 1.005200954758094, 'bef_v_on_time': 0.5317904786926791}


Looking at the results, we can see that the best parameters combination to detect the channel bursts with the above conditions are:

(**Dense Layer Weight**, **du**, **dv**):
- `(0.5, 0.95, 0.05)`
Followed by:
- `(0.5, 0.05, 0.95)`

Let's choose the parameters:
- `spike_weight = 0.5`
- `du = 0.95`
- `dv = 0.05`
  
as the optimal parameters and test it in a LAVA LIF Layer.